In [1]:
import numpy as np
import pandas as pd
import urllib2
import os
import progressbar
import urllib

In [24]:
data = pd.read_csv("P:/Projects/EU EU Sounds/Data/europeana-client/datasets/europeana_music_channel_metadata_mpeg.csv", 
                   engine="python", sep=";")
data.columns = ["europeana_id", "filelink"]

In [14]:
DATA_DIR = "E:/Data/MIR/EU_SOUND_CHANNEL"

In [15]:
import time
from functools import wraps


def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck, e:
                    msg = "%s, Retrying in %d seconds..." % (str(e), mdelay)
                    if logger:
                        logger.warning(msg)
                    else:
                        print msg
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry

In [17]:
#@retry(urllib2.URLError, tries=4, delay=3, backoff=2)
def downloadMP3(url, destination):
    
    req = urllib2.Request(url, None, headers={'User-Agent' : "Mozilla/5.0"})
    req2 = urllib2.urlopen(req)
    CHUNK = 16 * 1024
    with open(destination, 'wb') as fp:
        while True:
            chunk = req2.read(CHUNK)
            if not chunk:
                break
            fp.write(chunk)
            
downloadMP3(getProxyLink('/2059207/data_sounds_ADC_10', 'http://comhaltasarchive.ie/system/tracks/HQ/CL/ADC/preview-CL.ADC.10.mp3'),
            "D:/tmp/eu_sounds.mp3")

In [18]:
def getDestFullpath(europeana_id):
    
    tmp = europeana_id[1:].split("/")
    path = "%s/%s" % (DATA_DIR, tmp[0])
    filename = tmp[1].strip()
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    return "%s/%s.mp3" % (path, filename)
    
getDestFullpath('/09334/105568A205F62D456644AF24548CAE6CFF638D14')

'E:/Data/MIR/EU_SOUND_CHANNEL/09334/105568A205F62D456644AF24548CAE6CFF638D14.mp3'

In [19]:
def getProxyLink(europeana_id, link):
        
    return "http://proxy.europeana.eu%s?view=%s" % (europeana_id, urllib.quote(link))
    
getProxyLink('/2059207/data_sounds_ADC_10', 'http://comhaltasarchive.ie/system/tracks/HQ/CL/ADC/preview-CL.ADC.10.mp3')

'http://proxy.europeana.eu/2059207/data_sounds_ADC_10?view=http%3A//comhaltasarchive.ie/system/tracks/HQ/CL/ADC/preview-CL.ADC.10.mp3'

In [29]:

files_to_download = []

for i in range(data.shape[0]):
    
    row = data.loc[i]
    
    try:
        dest_path = getDestFullpath(row.europeana_id)

        if not os.path.exists(dest_path):
            files_to_download.append(i)
            
    except Exception as ex:
        print ex
        break

In [30]:
len(files_to_download)

1392

In [28]:
pbar = progressbar.ProgressBar()

num_ex = 0
#["europeana_id", "filelink"]
error_log = open("%s/error.2016-06-22.log" % (DATA_DIR), 'w')

for i in pbar(files_to_download):
    
    row = data.loc[i]
    
    try:
        dest_path = getDestFullpath(row.europeana_id)

        if not os.path.exists(dest_path):
            downloadMP3(getProxyLink(row.europeana_id, row.filelink), dest_path)
            
    except Exception as ex:
        #print row
        err_msg = "%s\t%s\t%s\n" % (row.europeana_id,row.filelink, ex)
        #print err_msg
        error_log.write(err_msg)
        #num_ex += 1
        #pass
        #break

    #if num_ex > 50:
    #    break
        
    if i % 1000 == 1000:
        #print i
        error_log.flush()
        
    i += 1
    
error_log.close()

100% (17459 of 17459) |#####################################################################################| Elapsed Time: 1 day, 1:53:34 Time: 1 day, 1:53:34


In [31]:
error_log.close()

In [32]:
e_log = pd.read_csv("E:/Data/MIR/EU_SOUND_CHANNEL/error.2016-06-22.log", sep="\t", header=None)
e_log.columns = ["europeana_id", "filelink", "error_msg"]

In [33]:
e_log.groupby("error_msg").count()

,europeana_id,filelink
error_msg,,
HTTP Error 400: Bad Request,7,7
HTTP Error 404: Not Found,1295,1295
HTTP Error 500: Internal Server Error,1,1
HTTP Error 502: Bad Gateway,2,2
HTTP Error 504: GATEWAY_TIMEOUT,87,87
